In [1]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
from os import listdir

# Fit folder
FitFolders = ["/Users/s2569857/Codes/NTK-interface/build/run/Results/test",]

In [ ]:
def NTK_frob (FitFolder):
    with open(FitFolder + "/meta.yaml", "r") as meta:
        Meta = yaml.load(meta, Loader=yaml.CLoader)
        nsteps = Meta["max_num_iterations"]

    step_size_to_store = 10
    NTK_frob_norms = np.zeros(int(nsteps /step_size_to_store))
    NTK_frob_norms_std = np.zeros(int(nsteps /step_size_to_store))
    time = np.array([t for t in range(1, nsteps, step_size_to_store)])
    nrep = 0

    for frep in [f.replace(".yaml", "") for f in sorted(listdir(FitFolder + "/log/"),  key=lambda s: int(s[s.find("_")+1:s.find(".")]))]:
        with open(FitFolder + "/log/" + frep + ".yaml", "r") as rep:
            Rep = yaml.load(rep, Loader=yaml.CLoader)
            for k,t in enumerate(range(1, nsteps, step_size_to_store)):
                NTK_frob_norms[k] += np.linalg.norm(Rep[t]["NTK"])
                NTK_frob_norms_std[k] += np.power(np.linalg.norm(Rep[t]["NTK"]), 2)
                
        nrep += 1

    NTK_frob_norms /= nrep
    NTK_frob_norms_std /= nrep
    NTK_frob_norms_std -= np.power(NTK_frob_norms, 2)

    return NTK_frob_norms, NTK_frob_norms_std

In [66]:
def first_derivative (FitFolder, tr_step, rep):
    with open(FitFolder + "/meta.yaml", "r") as meta:
        Meta = yaml.load(meta, Loader=yaml.CLoader)
        nsteps = Meta["max_num_iterations"]
    
    if tr_step > nsteps:
        raise Exception(f"The chosen time-step is too high. The maximum number of steps is {nsteps}")

    with open(FitFolder + "/log/replica_" + str(rep) + ".yaml", "r") as rep:
        Rep = yaml.load(rep, Loader=yaml.CLoader)

    size_of_data = len(Rep[tr_step]['dNN'].keys())
    size_of_output = Meta['NNarchitecture'][-1]
    number_of_derivatives = len(list(Rep[tr_step]['dNN'].values())[0]) - size_of_output
    derivative_tensor = np.ndarray((size_of_data, size_of_output, number_of_derivatives), dtype=float)     

    for id in range(size_of_data):
        for io in range(size_of_output):
            for ip in range(1, number_of_derivatives+1):
                derivative_tensor[id][io][ip-1] = list(Rep[tr_step]['dNN'].values())[id][io + ip * size_of_output]

    return derivative_tensor


def second_derivative (FitFolder, tr_step, rep):
    with open(FitFolder + "/meta.yaml", "r") as meta:
        Meta = yaml.load(meta, Loader=yaml.CLoader)
        nsteps = Meta["max_num_iterations"]
    
    if tr_step > nsteps:
        raise Exception(f"The chosen time-step is too high. The maximum number of steps is {nsteps}")

    with open(FitFolder + "/log/replica_" + str(rep) + ".yaml", "r") as rep:
        Rep = yaml.load(rep, Loader=yaml.CLoader)

    size_of_data = len(Rep[tr_step]['dNN'].keys())
    size_of_output = Meta['NNarchitecture'][-1]
    number_of_derivatives = len(list(Rep[tr_step]['dNN'].values())[0]) - size_of_output
    derivative_tensor = np.ndarray((size_of_data, size_of_output, number_of_derivatives, number_of_derivatives), dtype=float)     

    for id in range(size_of_data):
        for io in range(size_of_output):
            for ip1 in range(number_of_derivatives):
                for ip2 in range(1, number_of_derivatives+1):
                    derivative_tensor[id][io][ip1][ip2-1] = list(Rep[tr_step]['ddNN'].values())[id][ip1][io + ip2 * size_of_output]

    return derivative_tensor

In [67]:
df = first_derivative(FitFolders[0],0,0)
ddf = second_derivative(FitFolders[0],0,0)

In [75]:
df[0].shape

(1, 16)

In [74]:
ddf[1].shape

(1, 16, 16)

In [77]:
c = np.tensordot(df[0], ddf[1], axes=([1],[2]))